# Black-Scholes Option Pricing

In this notebook I will go through the basics of Black-Scholes derivative pricing,
using the case of European put and call options as a simple example.

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from black_scholes import call_european, put_european

%matplotlib inline

### Call option dependence on parameters

In the Black-Scholes model, the price of a call option depends on:
 1. $S$ : The price of the underlying
 2. $K$ : The strike price 
 3. $\sigma$ : The volatility 
 4. $T$ : Time to expiration
 5. $r$ : Risk-free rate of return
 6. $q$ : Continuous rate of dividend returns (if at all) 
 7. Discounted value of any dividend payouts (if at all) [Not yet implemented]
 
The basic formula is
$$ c = Se^{-qT}\Phi(d_1) - Ke^{-rT}\Phi(d_2) $$

where $\Phi$ is the cumulative distribution function of the unit normal distribution
$$ \Phi(z) \equiv \int_{-\infty}^z \frac{e^{-\frac{1}{2}x^2}}{\sqrt{2\pi}} {\rm d}x $$
and
$$ d_1 = \frac{1}{\sigma\sqrt{T}}\left[\ln(S/K) + \left(r-q+\frac{\sigma^2}{2}\right)T\right] $$
$$ d_2 = \frac{1}{\sigma\sqrt{T}}\left[\ln(S/K) + \left(r-q-\frac{\sigma^2}{2}\right)T\right] = d_1 - \sigma\sqrt{T}$$

It is straightforward to see that 
 - $\Delta \equiv \frac{\partial c}{\partial S} = \Phi(d_1)$, and
 - $\Phi(d_2)$ is the probability that the option will be exercised at maturity
 
*NOTE*: When we approach maturity, we can run into numerical problems if we try to naively evaluate this expression.

# Monte Carlo Approach

# Generating Brownian Motions